In [1]:
import os
from PIL import Image
import torch
import detect_mod
from pprint import pprint

detection_config = vars(detect_mod.parse_opt())

detection_model = detect_mod.warmup_model(weights=detection_config['weights'],
                                            data=detection_config['data'])
for key in ['weights', 'data', 'imgsz', 'device', 'half', 'dnn', 'source']:
    detection_config.pop(key)

def load_data(image_dir, label_dir):
    # Sorted list of image and label files
    image_files = sorted(os.listdir(image_dir))
    label_files = sorted(os.listdir(label_dir))

    images = []
    targets = []

    for image_file, label_file in zip(image_files, label_files):
        # Ensure corresponding pairs
        assert image_file.split('/')[-1][:-4] == label_file.split('/')[-1][:-4]

        # Load image
        image = Image.open(os.path.join(image_dir, image_file))
        image_width, image_height = image.size
        # image = torch.Tensor(image)  # Convert to tensor
        images.append(image)

        # Load labels
        with open(os.path.join(label_dir, label_file), 'r') as f:
            labels = f.readlines()
            boxes = []
            classes = []
            for label in labels:
                cls, x, y, w, h = map(float, label.split())
                # Convert normalized coordinates to original size
                x = x * image_width
                y = y * image_height
                w = w * image_width
                h = h * image_height
                # Convert to x1, y1, x2, y2 format
                x1 = x - w / 2
                y1 = y - h / 2
                x2 = x + w / 2
                y2 = y + h / 2
                boxes.append([x1, y1, x2, y2])
                classes.append(int(cls))

            targets.append({
                    "boxes": torch.tensor(boxes),
                    "labels": torch.tensor(classes)
                })
    
    return images, targets

# Load your data
images, targets = load_data("./runs/experiment/original/image/", "./runs/experiment/original/label/")
# detections = detect_mod.run_detection(detection_model, images, **detection_config)
# pprint(detections)


usage: ipykernel_launcher.py [-h] [--weights WEIGHTS [WEIGHTS ...]]
                             [--source SOURCE] [--data DATA]
                             [--imgsz IMGSZ [IMGSZ ...]]
                             [--conf-thres CONF_THRES] [--iou-thres IOU_THRES]
                             [--max-det MAX_DET] [--device DEVICE]
                             [--view-img] [--save-txt] [--save-conf]
                             [--save-crop] [--nosave]
                             [--classes CLASSES [CLASSES ...]]
                             [--agnostic-nms] [--augment] [--visualize]
                             [--update] [--project PROJECT] [--name NAME]
                             [--exist-ok] [--line-thickness LINE_THICKNESS]
                             [--hide-labels] [--hide-conf] [--half] [--dnn]
                             [--vid-stride VID_STRIDE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9021 --control=9019 --hb=9018 --Session.signature_sc

SystemExit: 2

In [2]:
%tb

SystemExit: 2